In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

In [2]:
import os
import re

# load file with itemIDs of the evaluation books
eval_ = pd.read_csv('data/evaluation.csv', delimiter='\|', engine='python')

In [3]:
# load amazon recommendations
amazon = pd.read_csv('data/amazon_recommendations.csv', sep='~')
amazon = amazon[['itemID', 'author', 'title', 'rec1_ID', 'rec2_ID', 'rec3_ID', 'rec4_ID', 'rec5_ID', 'rec6_ID', 'rec7_ID']]

In [4]:
# test book for computing the accuracy between own recommendations
# and the amazon recommendations
test_book_id = eval_.at[555, 'itemID']

In [5]:
print(test_book_id)
print(type(test_book_id))

10881
<class 'numpy.int64'>


In [6]:
# pseudo set of itemIDs of the own recommended books for the test book 10881
# dtype = np.int64()
recommendations = [np.int64(67938), np.int64(74953), np.int64(50640), np.int64(1234), np.int64(7763)]
recommendations

[67938, 74953, 50640, 1234, 7763]

In [7]:
def get_accuracy(book_id: np.int64, own_rec: list) -> float:
    # Keyword arguments:
    #
    # book_id -- ID of the test book (np.int64)
    # own_rec -- list of own recommendations for the test book (list elements of type np.int64())
    #
    # :return:
    # acc -- accuracy of the own recommendations compared to the amazon recommendations
    df = amazon[amazon['itemID'] == book_id]
    df = df[['rec1_ID', 'rec2_ID', 'rec3_ID', 'rec4_ID', 'rec5_ID', 'rec6_ID', 'rec7_ID']]
    vals = df.values.tolist()[0]
    vals = [v for v in vals if not pd.isnull(v) and v not in ['-', '---']]

    amazon_rec = []

    for v in vals:
        if type(v) == int:
            amazon_rec.append(v)
        if type(v) == str:
            numbers = re.findall('\d{2,}', v)
            for num in numbers:
                amazon_rec.append(int(num))
    
    own_rec = [(r in amazon_rec) for r in own_rec]

    acc = float(sum(own_rec) / len(own_rec)) * 100

    return acc

In [8]:
acc = get_accuracy(book_id=test_book_id, own_rec=recommendations)
print('accuracy for book {0} == {1} %'.format(test_book_id, round(acc, 2)))

accuracy for book 10881 == 60.0 %


In [9]:
# to get the overall accuracy loop over all 1000 ID of the eval_ file
# and average the accuracies